In [1]:
import pandas as pd
import gurobipy as grb
from gurobipy import GRB
import pickle

In [2]:
### filepaths
primary_facility_path = '../Data/Primary_Facility_50.csv'
secondary_facility_path = '../Data/Secondary_Facility_500.csv'
supplier_path = '../Data/Supplier.csv'
demand_path = '../ResearchData/1.Demand_Distribution.csv'
distance_path = '../Data/DISTANCE_DICT_VP50_VQ_500.pkl'

In [3]:
# fetch input data
p_df = pd.read_csv(primary_facility_path)
q_df = pd.read_csv(secondary_facility_path)
s_df = pd.read_csv(supplier_path)
demand_df = pd.read_csv(demand_path)

q_df['Index'] = q_df.index
s_df['Index'] = s_df.index
p_df['Index'] = p_df.index
demand_df['Index'] = demand_df.index

with open(distance_path, 'rb') as fp:
    DISTANCE_DICT = pickle.load(fp) 

In [4]:
## SETS
PRIMARY_FACILITY = tuple(p_df['Index'])
SECONDARY_FACILITY =  tuple(q_df['Index'])
CUSTOMER = tuple(demand_df['Index'])
SUPPLIER = tuple(s_df['Index'])


In [5]:
## VARIABLES
SUPPLIER_TO_PRIMARY = 'Xsp'
PRIMARY_TO_SECONDARY = 'Ypq'
PRIMARY_TO_PRIMARY = 'Tpp'
SECONDARY_TO_CUSTOMER = 'Zqk'
FACILITY = 'G'



In [6]:
## PARAMETERS
DEMAND = tuple(demand_df['Demand'])
DISTANCE = DISTANCE_DICT

FIXED_COST_PRIMARY = 250000
FIXED_COST_SECONDARY = 0.035
PACKAGING_COST_PRIMARY = 25000
PACKAGING_COST_SECONDARY = 0.140

R_PRIMARY = 2
R_SECONDARY = 50
C_SUPPLIER = 0.0006
C_PRIMARY = 0.0024
C_CUSTOMER = 0.012



In [7]:
DEMAND_DICT = dict(zip(CUSTOMER, DEMAND))

In [8]:
# model
model = grb.Model("MILP")

Restricted license - for non-production use only - expires 2026-11-23


In [9]:
# Define decision variables
Xsp_var = {(s, p): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=SUPPLIER_TO_PRIMARY + str(s) + '_' + str(p)) for s in SUPPLIER for p in PRIMARY_FACILITY}

Ypq_var = {(p, q): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=PRIMARY_TO_SECONDARY + str(p) + '_' + str(q)) for p in PRIMARY_FACILITY for q in SECONDARY_FACILITY}

Tpp_var = {(p1, p2): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=PRIMARY_TO_PRIMARY + str(p1) + '_' + str(p2)) for p1 in PRIMARY_FACILITY for p2 in set(PRIMARY_FACILITY) - set([p1])}

Zqk_var = {(q, k): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=SECONDARY_TO_CUSTOMER + str(q) + '_' + str(k)) for q in SECONDARY_FACILITY for k in CUSTOMER}

G_var = {('p', p): model.addVar(vtype=grb.GRB.BINARY, name=FACILITY + 'p_' + str(p)) for p in PRIMARY_FACILITY}
G_var.update({('q', q): model.addVar(vtype=grb.GRB.BINARY, name=FACILITY + 'q_' + str(q)) for q in SECONDARY_FACILITY})

# L = model.addVar(vtype=GRB.CONTINUOUS, name="L")
L = 2

In [10]:
## Add constraints for R_P and R_Q
model.addConstr(grb.quicksum(G_var[('p', p)] for p in PRIMARY_FACILITY) == R_PRIMARY)
model.addConstr(grb.quicksum(G_var[('q', q)] for q in SECONDARY_FACILITY) == R_SECONDARY)


<gurobi.Constr *Awaiting Model Update*>

In [11]:
for p in PRIMARY_FACILITY:
    model.addConstr(grb.quicksum(Xsp_var[(s, p)] for s in SUPPLIER) <= grb.quicksum(DEMAND_DICT[k] * G_var[('p', p)] for k in CUSTOMER))
    model.addConstr(grb.quicksum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY) <= grb.quicksum(DEMAND_DICT[k] * G_var[('p', p)] for k in CUSTOMER))

for q in SECONDARY_FACILITY:
    model.addConstr(grb.quicksum(Ypq_var[(p, q)] for p in PRIMARY_FACILITY) <= grb.quicksum(DEMAND_DICT[k] * G_var[('q', q)] for k in CUSTOMER))
    model.addConstr(grb.quicksum(Zqk_var[(q, k)] for k in CUSTOMER) <= grb.quicksum(DEMAND_DICT[k] * G_var[('q', q)] for k in CUSTOMER))
    

In [12]:
for p in PRIMARY_FACILITY:
    model.addConstr(grb.quicksum(Tpp_var[(p, p_)] for p_ in set(PRIMARY_FACILITY)-set([p])) <= grb.quicksum(DEMAND_DICT[k] * G_var[('p', p)] for k in CUSTOMER))

for p_ in PRIMARY_FACILITY:
    model.addConstr(grb.quicksum(Tpp_var[(p, p_)] for p in set(PRIMARY_FACILITY)-set([p_])) <= grb.quicksum(DEMAND_DICT[k] * G_var[('p', p)] for k in CUSTOMER))


In [13]:
for p in PRIMARY_FACILITY:
    model.addConstr(grb.quicksum(Xsp_var[(s, p)] for s in SUPPLIER) + grb.quicksum(Tpp_var[(p_, p)] for p_ in set(PRIMARY_FACILITY)-set([p])) == grb.quicksum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY) + grb.quicksum(Tpp_var[(p, p_)] for p_ in set(PRIMARY_FACILITY)-set([p])))

In [14]:
for q in SECONDARY_FACILITY:
    model.addConstr(grb.quicksum(Ypq_var[(p, q)] for p in PRIMARY_FACILITY) == grb.quicksum(Zqk_var[(q, k)] for k in CUSTOMER))

In [15]:
for p in PRIMARY_FACILITY:
    model.addConstr(grb.quicksum(Tpp_var[(p_, p)] for p_ in set(PRIMARY_FACILITY)-set([p])) == L * grb.quicksum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY))

In [16]:
for k in CUSTOMER:
    model.addConstr(grb.quicksum(Zqk_var[(q, k)] for q in SECONDARY_FACILITY) == DEMAND_DICT[k])

In [17]:
model.ModelSense = GRB.MINIMIZE
# model.setParam(GRB.Param.PoolSolutions, 100)
model.setParam('Method', 0) 


Set parameter Method to value 0


In [18]:
# Assuming 'model' is your Gurobi model
# constraints = model.getConstrs()
# constraints

In [19]:
objective = grb.quicksum(C_SUPPLIER * DISTANCE_DICT['SP'][f'XSP_{s}_{p}'] * Xsp_var[(s, p)] for s in SUPPLIER for p in PRIMARY_FACILITY) + \
                grb.quicksum(C_PRIMARY * DISTANCE_DICT['PQ'][f'XPQ_{p}_{q}'] * Ypq_var[(p, q)] for p in PRIMARY_FACILITY for q in SECONDARY_FACILITY) + \
                grb.quicksum(C_PRIMARY * DISTANCE_DICT['PP_'][f'XPP_{p}_{p_}'] * Tpp_var[(p, p_)] for p in PRIMARY_FACILITY for p_ in set(PRIMARY_FACILITY)-set([p])) + \
                grb.quicksum(C_CUSTOMER * DISTANCE_DICT['QK'][f'XQK_{q}_{k}'] * Zqk_var[(q, k)] for q in SECONDARY_FACILITY for k in CUSTOMER) + \
                grb.quicksum(FIXED_COST_PRIMARY * G_var[('p', p)] for p in PRIMARY_FACILITY) + \
                grb.quicksum(FIXED_COST_SECONDARY * G_var[('q', q)] for q in SECONDARY_FACILITY) + \
                grb.quicksum(PACKAGING_COST_PRIMARY * (grb.quicksum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY) + grb.quicksum(Tpp_var[(p, p_)] for p_ in set(PRIMARY_FACILITY)-set([p]))) for p in PRIMARY_FACILITY) + \
                grb.quicksum(PACKAGING_COST_SECONDARY * (grb.quicksum(Zqk_var[(q, k)] for k in CUSTOMER)) for q in SECONDARY_FACILITY)

            

In [20]:
model.setObjectiveN(objective, 100)


In [21]:
model.write('../gurobi_exps/MILP_gurobi.lp')


In [22]:
# Get the number of variables
num_variables = len(model.getVars())

# Get the number of constraints
num_constraints = len(model.getConstrs())

print(num_constraints, num_variables)

2173 213550


In [23]:
# Var : 213550
# Constraints : 2173

In [24]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
Method  0



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information